In [22]:
import psycopg2
import logging

logging.basicConfig(level=logging.INFO)

In [23]:
postgres_conn_ours = psycopg2.connect(
    host='127.0.0.1',
    user='postgres',
    password='16340954',
    database='bd_lv'
)

postgres_conn_m = psycopg2.connect(
    host='127.0.0.1',
    user='postgres',
    password='16340954',
    database='lv_m'
)

postgres_conn_a = psycopg2.connect(
    host='127.0.0.1',
    user='postgres',
    password='16340954',
    database='lv_a'
)

postgres_conn_f = psycopg2.connect(
    host='127.0.0.1',
    user='postgres',
    password='16340954',
    database='lv_f'
)

postgres_connections = [postgres_conn_ours, postgres_conn_m, postgres_conn_a, postgres_conn_f]

In [24]:
extract_ours = ('''
  SELECT *
  FROM categoria;
             
  SELECT *
  FROM cliente;
             
  SELECT *
  FROM locacao;
             
  SELECT *
  FROM patio;
             
  SELECT *
  FROM reserva;
             
  SELECT *
  FROM veiculo;
''')

extract_m = ('''
  SELECT *
  FROM cliente;
             
  SELECT *
  FROM grupo_categoria;
             
  SELECT *
  FROM locacao;
             
  SELECT *
  FROM patio;
             
  SELECT *
  FROM reserva;
             
  SELECT *
  FROM veiculo;
''')

extract_a = ('''
  SELECT *
  FROM clientes;
             
  SELECT *
  FROM endereco;
             
  SELECT *
  FROM grupos_veiculos;
             
  SELECT *
  FROM locacoes;
             
  SELECT *
  FROM prontuario;
             
  SELECT *
  FROM patios;
             
  SELECT *
  FROM reservas;
             
  SELECT *
  FROM veiculos;
''')

extract_f = ('''
  SELECT *
  FROM cliente;
             
  SELECT *
  FROM condutor;
             
  SELECT *
  FROM locacao;
             
  SELECT *
  FROM patio;
             
  SELECT *
  FROM reserva;
             
  SELECT *
  FROM veiculo;
''')

extract_queries = [extract_ours, extract_m, extract_a, extract_f]


In [29]:
def extract(postgres_connection, database_extract_query):
    try:
        cursor = postgres_connection.cursor()
        cursor.execute(database_extract_query)
        results = cursor.fetchall()
        print(results)
    except Exception as e:
        logging.error("An error occurred: %s", e)
        logging.debug("Failed query: %s", database_extract_query)
        postgres_connection.rollback()  # Roll back the transaction
    finally:
        cursor.close()  # Ensure the cursor is closed even if an error occurs

    return results

In [28]:
for connection, query in zip(postgres_connections, extract_queries):
   extract(connection, query)

[]
[]
[]
[]
